In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from config import *
from load_data import fetch_data, fetch_trajectory, fetch_player_box
from match_rounds import find_match_round_dtw, find_match_round_dtw_kmp
from Extractor.utils import compute_stat, interplate_and_align

# DataLoader

In [ ]:
import os

file_list = os.listdir(DATA_DIR)
file_list = list(filter(lambda x: "AD" in x, file_list))
file_list

In [ ]:
drop_list = ['pingpang.csv', 'tennis.csv', '.DS_Store', 'ControlGroupInfo.xlsx',]

In [ ]:
all_data = fetch_data(DATA_DIR, file_list, drop_list)
ball_data = fetch_trajectory(DATA_DIR)
player_box_data = fetch_player_box(os.path.join(DATA_DIR, "PlayerDetectionRes"))

# Feature Extraction

## Saccade Feature

In [ ]:
def compute_saccade_path_lite(df:pd.DataFrame):
    _speeds = [0]
    _angles = [0]
    indices = list(df.index)
    _dura = (1 * EYE_SAMPLE_TIME ) / 1000.0
    df["Screen.x"] = df["Screen.x"] * VR_SCALE
    df["Screen.y"] = df["Screen.y"] * VR_SCALE

    for _i in range(1, len(indices)):
        row_1 = df.loc[indices[_i - 1], :]
        row_2 = df.loc[indices[_i], :]

        _dist = np.sqrt((row_1["Screen.x"] - row_2["Screen.x"])**2 + (row_1["Screen.y"] - row_2["Screen.y"])**2)
        _angle = np.arctan(_dist / VR_ZDIST) / np.pi * 180

        _angles.append(_angle)
        _speeds.append(np.divide(_angle, _dura))

    return _speeds, _angles

## Trajectory Features

In [ ]:

def extract_saccade_features_lite(round_index, data:pd.DataFrame, ball_data_df:pd.DataFrame):
    _fea = {}

    _speeds, _angles = compute_saccade_path_lite(data)
    _fea = {**_fea, **compute_stat("SaccadeSpeed", _speeds)}
    _fea = {**_fea, **compute_stat("SaccadeAngel", _angles)}

    round_start_index = ball_data_df[ball_data_df["round"]==round_index].index[0]
    round_dura = ball_data_df[ball_data_df["round"]==round_index].shape[0]
    eye_start_index = data.index[0]
    _fea["SaccadeDelay"] = (eye_start_index-round_start_index) * EYE_SAMPLE_TIME
    _fea["SaccadeDelayPercent"] = ((eye_start_index-round_start_index) / round_dura) * EYE_SAMPLE_TIME

    # _fea["SaccadeTimeShift"] = (np.argmax(_speeds) - (data.shape[0] // 2)) * EYE_SAMPLE_TIME

    # bias_res = compute_drop_point_bias_pred(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasPred_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasPred_Final"] = bias_res[1] * VR_SCALE

    # bias_res = compute_drop_point_bias_delay(
    #     eye_data=data,
    #     eye_index=data.index[np.argmax(_speeds)],
    #     ball_data=ball_data_df,
    #     ball_round=round_index
    # )
    # _fea["SaccadeDropBiasDelay_Init"] = bias_res[0] * VR_SCALE
    # _fea["SaccadeDropBiasDelay_Final"] = bias_res[1] * VR_SCALE
    return _fea

In [ ]:
from Extractor.utils import compute_dtw

def extract_trajectory_lite(round_index, data:pd.DataFrame, ball_data_df):
    _fea = {}
    
    _ball_df = ball_data_df[ball_data_df["round"]==round_index]
    _dtw = compute_dtw(
            line_1=data.loc[:, ["Screen.x", "Screen.y"]],
            line_2=_ball_df.loc[:, ["Ball.x", "Ball.y"]],
            # scale_to_percentage=True,
            # scale_metrics="start"
        )
    
    _fea["TrajectoryDTW"] = _dtw

    return _fea

## Main extrat features function

In [ ]:
def extract_features_round(rounds:dict, data_df:pd.DataFrame, ball_data_df:pd.DataFrame):
    data_df["frame"] = data_df.index
    aligned_df = interplate_and_align(data_df, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    res = {}
    for round_index, round_indices in rounds.items():
        round_res = extract_saccade_features_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        
        traj_res = extract_trajectory_lite(
            round_index,
            aligned_df.loc[round_indices, ["Screen.x", "Screen.y"]],
            aligned_df.loc[:, ["Ball.x", "Ball.y", "round"]]
            )
        res[round_index] = {**round_res, **traj_res}
    return res

In [ ]:
from Extractor.utils import max_circle_radius

def threshold_find_match_round_dtw(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, order, scale_raw_data, mode="fast", dtw_th=1, dist_th=10):
    if mode=="fast":
        rounds, dtw_res = find_match_round_dtw_kmp(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)
    elif mode=="greedy":
        rounds, dtw_res = find_match_round_dtw(eye_data_df=eye_data, ball_data_df=ball_data_df, order=order, scale_raw_data=scale_raw_data)

    res = {}
    for _round, _round_index in rounds.items():
        if (max_circle_radius(eye_data.loc[_round_index, :]) >= dist_th) and (dtw_res[_round] <= dtw_th):
            res[_round] = _round_index

    return res, rounds

In [ ]:
def judge_inbox(row):
    TOL = 0
    _in_x = (row["Screen.x"] <= row["RightBottom.x"]+TOL) and (row["Screen.x"] >= row["LeftUp.x"]-TOL)
    _in_y = (row["Screen.y"] <= row["LeftUp.y"]+TOL) and (row["Screen.y"] >= row["RightBottom.y"]-TOL)
    return 1 if (_in_x and _in_y) else 0

def compute_inbox_dist(row):
    circle_x = (row["RightBottom.x"] + row["LeftUp.x"]) / 2
    circle_y = (row["LeftUp.y"] + row["RightBottom.y"]) / 2
    return np.sqrt((row["Screen.x"] - circle_x)**2 + (row["Screen.y"] - circle_y)**2) * VR_SCALE 


def extract_features_whole(eye_data:pd.DataFrame, ball_data_df:pd.DataFrame, player_box_data:dict):
    res = {}
    
    p1_bbox_df = pd.DataFrame(player_box_data["Player-1"])
    p2_bbox_df = pd.DataFrame(player_box_data["Player-2"])

    eye_data["frame"] = eye_data.index
    # aligned_df = interplate_and_align(eye_data, ball_data_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)
    aligned_p1_df = interplate_and_align(eye_data, p1_bbox_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)
    aligned_p2_df = interplate_and_align(eye_data, p2_bbox_df, EYE_SAMPLE_RATE, VIDEO_FPS, convert_dist=False)

    aligned_p1_df["inbox"] = aligned_p1_df.apply(judge_inbox, axis=1)
    aligned_p2_df["inbox"] = aligned_p2_df.apply(judge_inbox, axis=1)
    aligned_p1_df["box_dist"] = aligned_p1_df.apply(compute_inbox_dist, axis=1)
    aligned_p2_df["box_dist"] = aligned_p2_df.apply(compute_inbox_dist, axis=1)
    
    res["Player1AttentionRatio"] = aligned_p1_df["inbox"].sum() / aligned_p1_df.shape[0]
    res["Player2AttentionRatio"] = aligned_p2_df["inbox"].sum() / aligned_p2_df.shape[0]
    res["Player1MinToCircle"] = aligned_p1_df["box_dist"].min()
    res["Player2MinToCircle"] = aligned_p2_df["box_dist"].min()

    return res

In [ ]:
def extract_features(data, ball_data, player_box_data):
    data_df = pd.DataFrame(data).T
    data_df.ffill(inplace=True)
    data_df.bfill(inplace=True)

    ball_data_df = pd.DataFrame(ball_data)
    
    # match_rounds = label_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)

    match_rounds, rounds = threshold_find_match_round_dtw(data_df.copy(), ball_data_df.copy(), order=0, scale_raw_data=True, mode="fast")
    # match_rounds = find_match_round_hit(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, time_range=7, dist=300)
    saccade_features = extract_features_round(match_rounds, data_df.copy(), ball_data_df.copy())

    attention_features = extract_features_whole(data_df.copy(), ball_data_df.copy(), player_box_data)
    
    return {
        "match_rounds" : match_rounds, 
        "rounds": rounds,
        "saccade_fea": saccade_features,
        "attention_fea": attention_features
        }

    # saccade_features = extract_saccade_features(match_rounds, data_df.loc[:, ["Screen.x", "Screen.y"]], video_id)
    # trajectory_features = extract_trajectory(data_df.loc[:, ["Screen.x", "Screen.y"]], video_id, scale_to_percentage=True)

    # return {**saccade_features, **trajectory_features}

# Extract Features of all Participants

In [ ]:
all_people_fea = {}
all_people_rounds = {}
all_people_match_rounds = {}
video_list= []

for _person in all_data.keys():
    if not _person == "24071512_AD": continue
    _person_fea = {}
    _person_match_rounds = {}
    _person_rounds = {}

    for _video in all_data[_person].keys():
        if not _video == "p7": continue
        if not _video in video_list: video_list.append(_video)
        
        res = extract_features(
                data=all_data[_person][_video], 
                ball_data=ball_data[_video.split("_")[0]],
                player_box_data=player_box_data[_video.split("_")[0]]
            )
        
        _person_fea[_video] = {}
        _person_fea[_video]["All"] = {}
        _person_fea[_video]["All"].update(res["attention_fea"])
        _person_fea[_video]["All"].update({"MatchRoundRatio" : len(res["match_rounds"]) / len(res["rounds"])})
        _person_rounds[_video] = res["rounds"]
        if res["saccade_fea"]:
            _person_fea[_video].update(res["saccade_fea"])
            _person_match_rounds[_video] = res["match_rounds"]

    all_people_fea[_person] = _person_fea
    all_people_rounds[_person] = _person_rounds
    all_people_match_rounds[_person] = _person_match_rounds

In [ ]:
def single_video_res(people_fea_res, video_id):
    res = {}
    for _p, person_fea in people_fea_res.items():
        try:
            for _r, _fea in person_fea[video_id].items():
                res[f"{_p}-{_r}"] = _fea
        except:
            continue
    return res


In [ ]:
def single_person_res(people_fea_res, people_id):
    res = {}
    for _v, video_fea in people_fea_res[people_id].items():
        try:
            for _r, _fea in video_fea.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [ ]:
def single_person_rounds(people_rounds_res, people_id):
    res = {}
    for _v, video_rounds in people_rounds_res[people_id].items():
        try:
            for _r, _fea in video_rounds.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [ ]:
def single_person_match_rounds(people_match_rounds_res, people_id):
    res = {}
    for _v, video_rounds in people_match_rounds_res[people_id].items():
        try:
            for _r, _fea in video_rounds.items():
                res[f"{_v}-{_r}"] = _fea
        except:
            continue
    return res

In [ ]:
person_fea = single_person_res(people_fea_res=all_people_fea, people_id="24071512_AD")
# pd.DataFrame(person_fea).T.to_csv("24071512_AD_dtw_fast_rounds.csv")

In [ ]:
video_id = video_list[0]
print(video_id)
video_res = single_video_res(people_fea_res=all_people_fea, video_id=video_id)
pd.DataFrame(video_res).T

# Save Results to Output

In [ ]:
if not os.path.exists("output"):
    os.mkdir("output")

In [ ]:
import json
for file in file_list:
    if "Test" in file: continue
    people_id = file.split(".")[0]
    if not people_id in all_people_fea.keys(): continue
    person_fea = single_person_res(people_fea_res=all_people_fea, people_id=people_id)
    person_match_rounds = single_person_match_rounds(people_match_rounds_res=all_people_match_rounds, people_id=people_id)
    person_rounds = single_person_rounds(people_rounds_res=all_people_rounds, people_id=people_id)

    out_dir = f"output/{people_id}"
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    pd.DataFrame(person_fea).T.to_csv(f"{out_dir}/dtw_fast_features.csv")
    with open(f"{out_dir}/dtw_fast_rounds.json", "w") as f:
        json.dump(person_rounds, f)
    with open(f"{out_dir}/dtw_fast_match__rounds.json", "w") as f:
        json.dump(person_match_rounds, f)
